<a href="https://colab.research.google.com/github/KETI-saisc/new/blob/main/augmentation_bright2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
# import os
# file_path = "/content/drive/My Drive/Colab Notebooks/data/all/s2/test.pgm"
# if os.access(os.path.dirname(file_path), os.W_OK):
#     print("파일 쓰기 가능")
# else:
#     print("파일 쓰기 불가능")

In [2]:
import os
import cv2
import dlib
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

In [3]:
load_dir = "/content/drive/My Drive/Colab Notebooks/data/all/"
save_dir = "/content/drive/My Drive/Colab Notebooks/data/test/"

folders = ["s1", "s2", "s3", "s4", "s5", "s6", "s7"]

# 폴더 생성
for folder in folders:
    folder_path = os.path.join(save_dir, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [70]:
# img = cv2.imread(load_dir+"s1/1.pgm")

In [4]:
def random_bright(folder, folder2, file):
    filename = folder + "/" + file
    # loading in the images
    image = imageio.imread(filename)

    flip_vr=iaa.Flipud(p=1.0)
    flip_vr_image= flip_vr.augment_image(image)

    denoised = cv2.fastNlMeansDenoising(image, None, 5, 7, 21)
    image = denoised
    # contrast=iaa.GammaContrast((0.5, 1.5))
    # contrast_image =contrast.augment_image(image)
    # save_filename = folder + "/" + file.split(".")[0] + "_gamma1.pgm"
    # cv2.imwrite(save_filename, contrast_image)

    contrast=iaa.Multiply((0.5, 1.0))
    contrast_image =contrast.augment_image(image)
    save_filename = folder2 + "/" + file.split(".")[0] + "_dark.pgm"
    cv2.imwrite(save_filename, contrast_image)
    print (save_filename)

    contrast=iaa.Multiply((1.0, 1.5))
    contrast_image =contrast.augment_image(image)
    save_filename = folder2 + "/" + file.split(".")[0] + "_bright.pgm"
    cv2.imwrite(save_filename, contrast_image)
    print (save_filename)

    # contrast=iaa.Multiply((1.1, 1.2))
    # contrast_image =contrast.augment_image(image)
    # save_filename = folder2 + "/" + file.split(".")[0] + "_b2.pgm"
    # cv2.imwrite(save_filename, contrast_image)
    # print (save_filename)

    # contrast=iaa.Multiply((1.2, 1.3))
    # contrast_image =contrast.augment_image(image)
    # save_filename = folder2 + "/" + file.split(".")[0] + "_b3.pgm"
    # cv2.imwrite(save_filename, contrast_image)
    # print (save_filename)

    # contrast=iaa.Multiply((1.3, 1.4))
    # contrast_image =contrast.augment_image(image)
    # save_filename = folder2 + "/" + file.split(".")[0] + "_b4.pgm"
    # cv2.imwrite(save_filename, contrast_image)
    # print (save_filename)

In [5]:
def bright_normalization(folder, folder2,file):
    filename = folder + "/" + file
    # loading in the images
    image = imageio.imread(filename)

    mean_value = image.mean()

    # 평균 명도값을 100으로 맞추기
    alpha = 100 / mean_value
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=0)

    # 이미지 저장
    save_filename = folder2 + "/" + file.split(".")[0] + "_norm.pgm"
    cv2.imwrite(save_filename, adjusted)
    print (save_filename)

In [6]:
def face_feature_extraction(folder, folder2, file):

    filename = folder + "/" + file
    gray = imageio.imread(filename)

    # 얼굴 인식기 초기화
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("/content/drive/My Drive/Colab Notebooks/model/shape_predictor_68_face_landmarks.dat")

    # 얼굴 검출
    rects = detector(gray, 0)

    # 얼굴 특징점 추출
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        for i in range(68):
            x = shape.part(i).x
            y = shape.part(i).y
            cv2.circle(gray, (x, y), 2, (0, 255, 0), -1)

    # 이미지 저장
    save_filename = folder2 + "/" + file.split(".")[0] + "_feat.pgm"
    cv2.imwrite(save_filename, gray)
    print (save_filename)

In [7]:
import numpy as np
from skimage import io
from sklearn.preprocessing import StandardScaler

def image_standardization(folder, folder2, file):

    # 이미지 파일 경로와 파일명 설정
    # file_path = "path/to/image/file"
    # file_name = "image.jpg"

    file_path = folder
    file_name = file

    # gray = imageio.imread(filename)

    # 이미지 파일 불러오기
    image = io.imread(file_path + '/' + file_name, as_gray=True)

    # 이미지 데이터를 1차원 배열로 변환
    image_flat = np.ravel(image)

    # StandardScaler 객체 생성
    scaler = StandardScaler()

    # StandardScaler를 사용하여 데이터 standardization 수행
    image_scaled = scaler.fit_transform(image_flat.reshape(-1, 1))

    # standardization된 이미지를 다시 2차원 배열로 변환
    image_scaled = np.reshape(image_scaled, (92, 112))

    save_filename = folder2 + "/" + file.split(".")[0] + "_stand.pgm"
    cv2.imwrite(save_filename, image_scaled)
    print (save_filename)


In [8]:
# 폴더 이름 가져오기
face_id_list = os.listdir(load_dir)
# print (face_id)

# 파일 이름 가져오기
for face_id in face_id_list:
      print (load_dir + face_id)
      face_list = os.listdir(load_dir + face_id)
      for face in face_list:

          # 랜덤 bright 부여
          random_bright(load_dir + face_id, save_dir + face_id,face)

          # # 같은 밝기로 변환
          # bright_normalization(load_dir + face_id, save_dir + face_id, face)

          # # 특징점 추출 (랜드마크 표기)
          # face_feature_extraction(load_dir + face_id, save_dir + face_id, face)

          # # Standardization
          # image_standardization(load_dir + face_id, save_dir + face_id,face)


/content/drive/My Drive/Colab Notebooks/data/all/s3


<ipython-input-4-4defd278ac48>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_40_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_40_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_42_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_42_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_41_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_41_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_15_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_15_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_26_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_26_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_43_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_43_bright.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_38_dark.pgm
/content/drive/My Drive/Colab Notebooks/data/test/s3/s3_38_bright.pgm
/content/drive/My Drive/Colab Note